# 9.7. 텐서플로우를 이용한 CNN (Convolutional Neural Network) 예제

여기에서는 CNN을 구축하고 학습시킴으로써, MNIST 데이터에 대해 얼마만큼 정확하게 맞추는 지 확인해 보도록 하겠습니다. 

POINT
* CNN 모델을 구축합니다.
* CNN 모델을 MNIST 데이터에 대해 학습시키고 평가합니다.

### 9.7.1. 필요한 모듈 불러오기

In [6]:
import tensorflow as tf

In [7]:
from tensorflow.examples.tutorials.mnist import input_data

In [8]:
mnist = input_data.read_data_sets("./data/mnist", one_hot=True)

Extracting ./data/mnist/train-images-idx3-ubyte.gz
Extracting ./data/mnist/train-labels-idx1-ubyte.gz
Extracting ./data/mnist/t10k-images-idx3-ubyte.gz
Extracting ./data/mnist/t10k-labels-idx1-ubyte.gz


MNIST 데이터에 대한 설명은 앞에서 상세히 하였으므로 여기에서는 생략합니다. 

### 9.7.2. CNN 모델 만들기

입력 계층을 플레이스홀더로 정의해 줍니다.

In [9]:
x_true = tf.placeholder(tf.float32, [None, 28, 28, 1])
y_true = tf.placeholder(tf.float32, [None, 10])

은닉 계층 1을 정의해 줍니다.
* (3,3) 크기를 가지는 필터를 설정합니다.
* Stride와 padding을 설정하여 convolution을 실행합니다.
* Non-linearity로 ReLU를 적용합니다. 
* Max pooling을 수행합니다. 
* 0.8의 확률로 dropout을 수행합니다.

In [10]:
weight_1 = tf.Variable(tf.truncated_normal([3,3,1,32], stddev=0.01))
hidden_1 = tf.nn.conv2d(x_true, weight_1, strides=[1,1,1,1], padding='SAME')
hidden_1 = tf.nn.relu(hidden_1)
hidden_1 = tf.nn.max_pool(hidden_1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
hidden_1 = tf.nn.dropout(hidden_1, keep_prob=0.8)

은닉 계층 2를 정의해 줍니다. 
* 절차는 은닉 계층 1과 대동소이합니다.

In [11]:
weight_2 = tf.Variable(tf.truncated_normal([3,3,32,64], stddev=0.01))
hidden_2 = tf.nn.conv2d(hidden_1, weight_2, strides=[1,1,1,1], padding='SAME')
hidden_2 = tf.nn.relu(hidden_2)
hidden_2 = tf.nn.max_pool(hidden_2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

FC (Fully Connected) 계층으로 가기 위해 hidden_2의 차원을 줄여줍니다.

In [12]:
hidden_2 = tf.reshape(hidden_2, [-1,7*7*64]) # -1은 나머지 전체를 의미
hidden_2 = tf.nn.dropout(hidden_2, keep_prob=0.8)

FC 계층을 정의해 줍니다. 
* 단순히 matmul() 연산을 수행합니다.
* ReLU와 dropout을 적용합니다.

In [13]:
weight_3 = tf.Variable(tf.truncated_normal([7*7*64, 256], stddev=0.01))
fc_1 = tf.matmul(hidden_2, weight_3)
fc_1 = tf.nn.relu(fc_1)
fc_1 = tf.nn.dropout(fc_1, 0.5)

최종 계층인 소프트맥스 계층을 정의해 줍니다. 

In [14]:
weight_4 = tf.Variable(tf.truncated_normal([256,10], stddev=0.01))
y_pred = tf.matmul(fc_1, weight_4)

cost와 optimizer를 정의해 줍니다.

In [15]:
cost = tf.nn.softmax_cross_entropy_with_logits(logits=y_pred, labels=y_true)
cost = tf.reduce_mean(cost)
optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
optimizer = optimizer.minimize(cost)

### 9.7.3. CNN 모델 학습시키기

In [16]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [17]:
batch_size = 100
num_batch = int(mnist.train.num_examples/batch_size)

실제 학습을 진행합니다. 
* 기본 신경망과는 비교가 안 되게 오랜 시간이 소요되니 주의하시기 바랍니다.

In [18]:
for epoch in range(10):
    total_cost = 0
    for i in range(num_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        # CNN 모델을 위한 자료형인 [28,28,1]의 형태로 reshaping
        batch_xs = batch_xs.reshape(-1,28,28,1)
        _, cost_value = sess.run([optimizer, cost], 
                           {x_true:batch_xs, y_true:batch_ys})
        total_cost += cost_value
    print "Epoch : {0}, Cost : {1}".format(epoch + 1, total_cost/num_batch)
print "최적화가 완료되었습니다."

Epoch : 1, Cost : 0.289691926761
Epoch : 2, Cost : 0.150766817182
Epoch : 3, Cost : 0.138363266345
Epoch : 4, Cost : 0.127491725536
Epoch : 5, Cost : 0.129692074318
Epoch : 6, Cost : 0.126229241216
Epoch : 7, Cost : 0.128616164236
Epoch : 8, Cost : 0.121198371507
Epoch : 9, Cost : 0.125352199122
Epoch : 10, Cost : 0.116573325987
최적화가 완료되었습니다.


### 9.7.4. 평가하기

정확도를 계산합니다. 

In [19]:
correct_prediction = tf.equal(tf.argmax(y_pred,1), tf.argmax(y_true,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print "정확도 : {0}".format(sess.run(accuracy, 
                                  {x_true:mnist.test.images.reshape(-1, 28, 28, 1), 
                                   y_true:mnist.test.labels}))

정확도 : 0.971700012684


불과 10번의 학습으로 97.1%라는 정확도를 얻은 것을 확인할 수 있었습니다. 